# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f84c8bfaeb0>
├── 'a' --> tensor([[ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566]])
└── 'x' --> <FastTreeValue 0x7f84c8bfadc0>
    └── 'c' --> tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                        [ 0.2159, -0.1493, -1.3037, -0.6723],
                        [ 1.2703, -1.4458, -0.0149, -0.2297]])

In [4]:
t.a

tensor([[ 0.4661,  0.4314, -0.5292],
        [ 0.2036, -0.5788,  1.3566]])

In [5]:
%timeit t.a

64.4 ns ± 0.0575 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f84c8bfaeb0>
├── 'a' --> tensor([[ 0.1921,  1.0337, -0.2224],
│                   [ 0.4762, -0.5636, -0.2172]])
└── 'x' --> <FastTreeValue 0x7f84c8bfadc0>
    └── 'c' --> tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                        [ 0.2159, -0.1493, -1.3037, -0.6723],
                        [ 1.2703, -1.4458, -0.0149, -0.2297]])

In [7]:
%timeit t.a = new_value

67.4 ns ± 0.0287 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566]]),
    x: Batch(
           c: tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                      [ 0.2159, -0.1493, -1.3037, -0.6723],
                      [ 1.2703, -1.4458, -0.0149, -0.2297]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4661,  0.4314, -0.5292],
        [ 0.2036, -0.5788,  1.3566]])

In [11]:
%timeit b.a

60.6 ns ± 0.0498 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.7910,  0.3611,  0.1427],
               [ 0.3189, -0.4867,  1.4846]]),
    x: Batch(
           c: tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                      [ 0.2159, -0.1493, -1.3037, -0.6723],
                      [ 1.2703, -1.4458, -0.0149, -0.2297]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

147 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f84a00a4520>
├── 'a' --> tensor([[[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]],
│           
│                   [[ 0.4661,  0.4314, -0.5292],
│                    [ 0.2036, -0.5788,  1.3566]]])
└── 'x' --> <FastTreeValue 0x7f84a018cfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8411e6e2e0>
├── 'a' --> tensor([[ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566],
│                   [ 0.4661,  0.4314, -0.5292],
│                   [ 0.2036, -0.5788,  1.3566]])
└── 'x' --> <FastTreeValue 0x7f8411e6e730>
    └── 'c' --> tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                        [ 0.2159, -0.1493, -1.3037, -0.6723],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 48.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]],
       
               [[ 0.4661,  0.4314, -0.5292],
                [ 0.2036, -0.5788,  1.3566]]]),
    x: Batch(
           c: tensor([[[ 0.2624,  0.5311,  0.0712, -1.0866],
                       [ 0.2159, -0.1493, -1.3037, -0.6723],
                       [ 1.2703, -1.4458, -0.0149, -0.2297]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 295 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566],
               [ 0.4661,  0.4314, -0.5292],
               [ 0.2036, -0.5788,  1.3566]]),
    x: Batch(
           c: tensor([[ 0.2624,  0.5311,  0.0712, -1.0866],
                      [ 0.2159, -0.1493, -1.3037, -0.6723],
                      [ 1.2703, -1.4458, -0.0149, -0.2297],
                      [ 0.2624,  0.5311,  0.0712, -1.0866],
                      [ 0.2159, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 428 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

335 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
